# Means analysis

of results

In [ ]:
%pylab inline

import pandas as pd
import seaborn as sns; sns.set()

pd.set_option('display.max_columns', None)

eval_res = pd.read_parquet('evaluation_results.parquet')
eval_res.head()

In [ ]:
df = pd.read_parquet('evaluation_results_new.parquet')
df.head()

In [ ]:
df[df.model == 'optuna_lgbm_direct'].sort_values(['cv_split', 'data_kind', 'precision_business'], ascending=False).set_index(['top-k',  'cv_split', 'data_kind',])[['precision', 'precision_business', 'tp_business', #'tn_business', 
'fp_business', #'fn_business'

'tp', 'fp', 
]].unstack()

## new analysis

Overfitting models om the training dataset:

In [ ]:
potentially_overfitting_models = df[df.precision == 1]['model'].unique()
potentially_overfitting_models

In [ ]:
df[df.precision == 1][['model', 'cv_split', 'data_kind']].drop_duplicates()

In [ ]:
df.shape

In [ ]:
df[df.model.isin(potentially_overfitting_models)][['cv_split', 'model', 'top-k', 'data_kind', 'precision', 'precision_business']]

In [ ]:
df[df.model == 'nnet_lstm_ss_train_cutmix_optuna_direct_02'][['cv_split', 'data_kind', 'precision', 'precision_business']]

> NOTICE: these overfitting models are not (and also were not) displayed in the summary we looked at last time.

Here the **business rule** (ranked) variant. Notice the low number of `FP`!

In [ ]:
d = df[df.data_kind != 'global'].groupby(['model', 'top-k', 'data_kind'])[['fbeta_1', 'fbeta_1_business', 'precision', 'precision_business', 'recall', 'recall_business', 'fp', 'fp_business', 'tp', 'tp_business', 'fn', 'fn_business']].mean().unstack()
d.sort_values([('precision_business', 'test')], ascending=False)

This is the raw classification of each individual device.

> NOTICE the high number of FP!

In [ ]:
d.sort_values([('precision', 'test')], ascending=False)

Both metrics (raw probability theresholded at 50% and the ranked business result) are **NOT** to be directly compared / related.

In [ ]:
# df_p = df
models_of_interest = [
    'largest_tx_change', 
    'nnet_lstm_cw', 
    #'nnet_TST_dropout_8', 
    #'nnet_TST_dropout_65',
    'nnet_TST_optuna_9',
    #'nnet_InceptionTime_cw',
    'nnet_InceptionTime_optuna',
    'nnet_lstm_ss_train', 
    'nnet_lstm_ss_ft', 
    'optuna_lgbm_direct', 'nnet_lstm_ss_train_cutmix_optuna', 'nnet_lstm_ss_train_cutmix_optuna_02',
    #'optuna_lgbm',
    'logistic_regression'
]

replacement_dict = {
    'largest_tx_change': 'business rule \n(largest tx change)',
    'nnet_lstm_cw' : 'LSTM',
    'nnet_TST_optuna_9': 'TST',
    'nnet_InceptionTime_optuna': 'InceptionTime',
    'nnet_lstm_ss_train': 'LSTM, \nself supervised (train)',
    'nnet_lstm_ss_ft': 'LSTM, \nself supervised (fine tuning)',
    'nnet_lstm_ss_train_cutmix_optuna': 'LSTM, \nself supervised (train) + \ndata augmentation',
    'optuna_lgbm': 'lightGBM \n(old different metric)',
    'optuna_lgbm_direct': 'lightGBM',
    'logistic_regression': 'logistic regression'
    
}

df_p = df[(df.model.isin(models_of_interest)) & (df.data_kind != 'train')]

df_p = df_p.rename(columns = {'fbeta_5': 'F-0.5', 'fbeta_1':'F-1', 'precision_business':'precision @k'})
df_p['model'] = df_p['model'].replace(replacement_dict)

df_p = df_p.rename(columns = {'fbeta_5': 'F-0.5', 'fbeta_1':'F-1', 'precision_business':'precision @k'})
df_p['model'] = df_p['model'].replace(replacement_dict)

eval_res = df_p

df_p_tt = df[(df.model.isin(models_of_interest)) & (df.data_kind.isin(['train', 'test']))]
df_p_tt = df_p_tt.rename(columns = {'fbeta_5': 'F-0.5', 'fbeta_1':'F-1', 'precision_business':'precision @k'})
df_p_tt['model'] = df_p_tt['model'].replace(replacement_dict)

df_p.model.unique()

In [ ]:
# eval_res[eval_res['top-k'] == 1]
eval_res_tbl = df_p.copy()
eval_res_tbl.model = eval_res_tbl.model.str.replace('\n', '')
tbl_res = eval_res_tbl[eval_res_tbl.model != 'largest tx change, full data'].drop(['cv_split', 'train_triggers', 'test_triggers', 'accuracy', 'tn', 'fn', 'fp',
                                                                                   'tp', 'F-0.5', 'recall_business', 'fbeta_5_business', 'fbeta_1_business',
                                                                                   
                                                                                   'F-1', #'recall',
                                                                                   
                                                                                   #'fp_business', 'tp_business',
                                                                                   
                                                                                   'tn_business', 'fn_business', 'accuracy_business'], axis=1).groupby(['top-k', 
                                          'model']).agg(['mean', 'std']).round(3).sort_values(['top-k', ('precision @k', 'mean')], ascending=[True, False])
tbl_res = tbl_res.rename(columns={'fp_business': 'false positives@k', 'tp_business': 'true positives@k', 'precision':'precision step 1', 'recall': 'recall step 1'})
tbl_res = tbl_res[['precision step 1', 'recall step 1', 'precision @k', 'false positives@k', 'true positives@k']]
tbl_res

In [ ]:
with open('table_rca_results_new_business.tex', "w+") as pt:
    # pt.write(tbl_res.to_latex(multirow=True, longtable=True, escape=False, encoding="UTF-8"))
    pt.write(tbl_res.to_latex(index=True))

## old analysis

In [ ]:
#eval_res = eval_res[(eval_res.model.isin([
#    'largest_tx_change_full_data', 'largest_tx_change', 'nnet_lstm_cw', 'nnet_TST_dropout_8', 'nnet_InceptionTime_cw', 
#    'nnet_lstm_ss_train', 'nnet_lstm_ss_ft',
#    #   'nnet_lstm_ss_train_cutmix', 
#    #'optuna_lgbm_direct', 
#    'nnet_lstm_ss_train_cutmix_optuna',
#    
#    #'lgb', 'lgb_small', 
#    'optuna_lgbm',
#])) #& (eval_res['top-k'] == 1)
#                   ]

In [ ]:
#eval_res['model'] = eval_res['model'].replace({
#    'largest_tx_change_full_data' :'largest tx change, \nfull data',
#    'largest_tx_change': 'largest tx change',
#    'nnet_lstm_cw' : 'LSTM',
#    'nnet_TST_dropout_8': 'TST',
#    'nnet_InceptionTime_cw': 'InceptionTime',
#    'nnet_lstm_ss_train': 'LSTM, \nself supervised (train)',
#    'nnet_lstm_ss_ft': 'LSTM, \nself supervised (fine tuning)',
#    'nnet_lstm_ss_train_cutmix_optuna': 'LSTM, \nself supervised (train) + \ndata augmentation',
#    'optuna_lgbm': 'lightGBM'
#    
#})

In [ ]:
#eval_res = eval_res.rename(columns = {'fbeta_5': 'F-0.5', 'fbeta_1':'F-1'})

In [ ]:
#eval_res.model.unique()

In [ ]:
## eval_res[eval_res['top-k'] == 1]
#eval_res_tbl = eval_res.copy()
#eval_res_tbl.model = eval_res_tbl.model.str.replace('\n', '')
#tbl_res = eval_res_tbl[eval_res_tbl.model != 'largest tx change, full data'].drop(['cv_split', 'train_triggers', 'test_triggers', 'accuracy', 'tn', 'fn', 'fp', 'tp', 'F-0.5'], axis=1).groupby(['top-k', 
#                                          'model']).agg(['mean', 'std']).round(3).sort_values(['top-k', ('F-1', 'mean')], ascending=[True, False])#
#
#tbl_res

In [ ]:
#with open('table_rca_results.tex', "w+") as pt:
#    # pt.write(tbl_res.to_latex(multirow=True, longtable=True, escape=False, encoding="UTF-8"))
#    pt.write(tbl_res.to_latex(index=True))

In [ ]:
#fs_axes = 18
#fs_lab = 20

sns.set(font_scale = 1.7)
#sns.set_context("paper", rc={"font.size":fs_lab,"axes.titlesize":fs_lab,"axes.labelsize":fs_axes}) 

In [ ]:
a4_dims = (11.7, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)
ax = sns.boxplot( y="recall", x='model',data=eval_res, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)

plt.tight_layout()
plt.xlabel('')
plt.savefig('root_cause_eval_recall_new_business_model_raw.pdf')
plt.show()

In [ ]:
a4_dims = (11.7, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)
ax = sns.boxplot( y="precision", x='model',data=eval_res, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.tight_layout()
plt.xlabel('')
plt.savefig('root_cause_eval_precision_new_business_model_raw.pdf')
plt.show()

In [ ]:
a4_dims = (11.7, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)
ax = sns.boxplot( y="precision @k", x='model', hue='top-k',data=df_p, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.tight_layout()
plt.xlabel('')
plt.savefig('root_cause_eval_precision_new_business.pdf')
plt.show()

## now test data evaluation

In [ ]:
df_p_tt.head()

In [ ]:
a4_dims = (11.7, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)
ax = sns.boxplot( y="precision", x='model',data=df_p_tt[df_p_tt['top-k'] == 1], hue='data_kind', ax=ax)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.tight_layout()
plt.xlabel('')
plt.title('top-k = 1')
plt.savefig('root_cause_eval_tt_precision_1.pdf')
plt.show()

In [ ]:
a4_dims = (11.7, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)
ax = sns.boxplot( y="precision", x='model',data=df_p_tt[df_p_tt['top-k'] == 3], hue='data_kind', ax=ax)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.tight_layout()
plt.xlabel('')
plt.title('top-k = 3')
plt.savefig('root_cause_eval_tt_precision_3.pdf')
plt.show()

In [ ]:
a4_dims = (11.7, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)
ax = sns.boxplot( y="recall", x='model',data=df_p_tt[df_p_tt['top-k'] == 1], hue='data_kind', ax=ax)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.tight_layout()
plt.xlabel('')
plt.title('top-k = 1')
plt.savefig('root_cause_eval_tt_recall_1.pdf')
plt.show()

In [ ]:
a4_dims = (11.7, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)
ax = sns.boxplot( y="recall", x='model',data=df_p_tt[df_p_tt['top-k'] == 3], hue='data_kind', ax=ax)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.tight_layout()
plt.xlabel('')
plt.title('top-k = 3')
plt.savefig('root_cause_eval_tt_recall_3.pdf')
plt.show()

## old things

In [ ]:
#a4_dims = (11.7, 8.27)
#fig, ax = pyplot.subplots(figsize=a4_dims)
#ax = sns.boxplot( y="F-1", x='model', hue='top-k',data=eval_res, ax=ax)
#ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
#plt.tight_layout()
##plt.savefig('root_cause_eval_fbeta_1.pdf')
#plt.show()

In [ ]:
#a4_dims = (11.7, 8.27)
#fig, ax = pyplot.subplots(figsize=a4_dims)
#ax = sns.barplot( y="F-1", x='model', hue='top-k',data=eval_res.groupby(['top-k','model'])['F-1'].median().reset_index(), ax=ax)
#ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
#plt.tight_layout()
#plt.show()

In [ ]:
#a4_dims = (11.7, 8.27)
#fig, ax = pyplot.subplots(figsize=a4_dims)
#ax = sns.boxplot( y="F-0.5", x='model', hue='top-k',data=eval_res, ax=ax)
#ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
#plt.tight_layout()
##plt.savefig('root_cause_eval_fbeta_5.pdf')
#plt.show()

In [ ]:
#a4_dims = (11.7, 8.27)
#fig, ax = pyplot.subplots(figsize=a4_dims)
#ax = sns.boxplot( y="accuracy", x='model', hue='top-k',data=eval_res, ax=ax)
#ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
#plt.tight_layout()
##plt.savefig('root_cause_eval_accuracy.pdf')
#plt.show()